# Advanced IGDB + PyTorch Recommender (Ratings 1-10)
This notebook builds a better recommender using ratings, genres and platforms.

In [18]:
import requests
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import random

In [ ]:

CLIENT_ID = ""
ACCESS_TOKEN = ""

headers = {
    "Client-ID": CLIENT_ID,
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

def fetch_games():
    url = "https://api.igdb.com/v4/games"
    query = "fields id,name,genres.name,platforms.name; limit 80;"
    response = requests.post(url, headers=headers, data=query)
    return response.json()

raw_games = fetch_games()
print("Fetched:", len(raw_games))
raw_games

Fetched: 80


[{'id': 350392,
  'genres': [{'id': 5, 'name': 'Shooter'}],
  'name': 'Rival Species',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 30492,
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'},
   {'id': 15, 'name': 'Strategy'},
   {'id': 31, 'name': 'Adventure'},
   {'id': 32, 'name': 'Indie'}],
  'name': 'Hearts of Chaos',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 339266,
  'name': 'Power Guy World',
  'platforms': [{'id': 19, 'name': 'Super Nintendo Entertainment System'}]},
 {'id': 371776,
  'genres': [{'id': 31, 'name': 'Adventure'},
   {'id': 32, 'name': 'Indie'},
   {'id': 34, 'name': 'Visual Novel'}],
  'name': 'Born for the Light: Wavelength Radiant Collapse',
  'platforms': [{'id': 6, 'name': 'PC (Microsoft Windows)'}]},
 {'id': 376989,
  'genres': [{'id': 10, 'name': 'Racing'}],
  'name': 'Dune Buggy',
  'platforms': [{'id': 82, 'name': 'Web browser'}]},
 {'id': 85727,
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'

In [20]:

records = []
for g in raw_games:
    genres = [x['name'] for x in g.get('genres', [])]
    platforms = [x['name'] for x in g.get('platforms', [])]
    records.append({"game_id": g["id"], "name": g["name"], "genres": genres, "platforms": platforms})

df_games = pd.DataFrame(records)
df_games.head()


,game_id,name,genres,platforms
0,350392,Rival Species,[Shooter],[PC (Microsoft Windows)]
1,30492,Hearts of Chaos,"[Role-playing (RPG), Strategy, Adventure, Indie]",[PC (Microsoft Windows)]
2,339266,Power Guy World,[],[Super Nintendo Entertainment System]
3,371776,Born for the Light: Wavelength Radiant Collapse,"[Adventure, Indie, Visual Novel]",[PC (Microsoft Windows)]
4,376989,Dune Buggy,[Racing],[Web browser]


In [21]:

genre_mlb = MultiLabelBinarizer()
platform_mlb = MultiLabelBinarizer()

genre_features = genre_mlb.fit_transform(df_games["genres"])
platform_features = platform_mlb.fit_transform(df_games["platforms"])

df_games_encoded = pd.concat([df_games,
                              pd.DataFrame(genre_features, columns=genre_mlb.classes_),
                              pd.DataFrame(platform_features, columns=platform_mlb.classes_)], axis=1)
df_games_encoded.head()


,game_id,name,genres,platforms,Adventure,Arcade,Fighting,Hack and slash/Beat 'em up,Indie,Music,...,Playdate,Sega Saturn,Super Famicom,Super Nintendo Entertainment System,Web browser,Wii,Wii U,Xbox One,Xbox Series X|S,iOS
0,350392,Rival Species,[Shooter],[PC (Microsoft Windows)],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30492,Hearts of Chaos,"[Role-playing (RPG), Strategy, Adventure, Indie]",[PC (Microsoft Windows)],1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,339266,Power Guy World,[],[Super Nintendo Entertainment System],0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,371776,Born for the Light: Wavelength Radiant Collapse,"[Adventure, Indie, Visual Novel]",[PC (Microsoft Windows)],1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,376989,Dune Buggy,[Racing],[Web browser],0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [22]:

users = list(range(1, 31))
ratings = []

for u in users:
    rated = random.sample(df_games["game_id"].tolist(), 10)
    for g in rated:
        ratings.append({"user_id": u, "game_id": g, "rating": random.randint(1, 10)})

df_ratings = pd.DataFrame(ratings)
df_ratings.head()


,user_id,game_id,rating
0,1,314671,3
1,1,333673,10
2,1,147850,10
3,1,359862,5
4,1,152887,8


In [33]:

user_enc = LabelEncoder()
game_enc = LabelEncoder()

df_ratings["user"] = user_enc.fit_transform(df_ratings["user_id"])
df_ratings["game"] = game_enc.fit_transform(df_ratings["game_id"])

num_users = df_ratings["user"].nunique()
num_games = df_ratings["game"].nunique()

print(num_users, num_games)


30 78


In [34]:

feature_cols = list(genre_mlb.classes_) + list(platform_mlb.classes_)
game_feature_matrix = df_games_encoded[feature_cols].values

game_features_tensor = torch.tensor(game_feature_matrix, dtype=torch.float32)
num_features = game_features_tensor.shape[1]


In [35]:

class RatingModel(nn.Module):
    def __init__(self, n_users, n_games, n_features, emb_dim=32):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.game_emb = nn.Embedding(n_games, emb_dim)
        self.feature_layer = nn.Linear(n_features, emb_dim)
        self.fc = nn.Sequential(
            nn.Linear(emb_dim*3, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, u, g, f):
        return self.fc(torch.cat([
            self.user_emb(u),
            self.game_emb(g),
            self.feature_layer(f)
        ], dim=1))

model = RatingModel(num_users, num_games, num_features)


In [36]:

X_users = torch.tensor(df_ratings["user"].values)
X_games = torch.tensor(df_ratings["game"].values)
y = torch.tensor(df_ratings["rating"].values, dtype=torch.float32)

features = game_features_tensor[X_games]

dataset = TensorDataset(X_users, X_games, features, y)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(20):
    total = 0
    for u, g, f, r in loader:
        optimizer.zero_grad()
        preds = model(u, g, f).squeeze()
        loss = loss_fn(preds, r)
        loss.backward()
        optimizer.step()
        total += loss.item()
    print("Epoch", epoch, "Loss", total)


Epoch 0 Loss 361.9013957977295
Epoch 1 Loss 161.30577421188354
Epoch 2 Loss 132.79821062088013
Epoch 3 Loss 113.00691533088684
Epoch 4 Loss 106.6854043006897
Epoch 5 Loss 91.10986018180847
Epoch 6 Loss 82.56047677993774
Epoch 7 Loss 67.29344916343689
Epoch 8 Loss 60.12333619594574
Epoch 9 Loss 54.34017539024353
Epoch 10 Loss 45.78586518764496
Epoch 11 Loss 39.929129511117935
Epoch 12 Loss 33.2413624227047
Epoch 13 Loss 27.20164304971695
Epoch 14 Loss 24.937470138072968
Epoch 15 Loss 18.869788989424706
Epoch 16 Loss 17.30458116531372
Epoch 17 Loss 13.820556074380875
Epoch 18 Loss 12.529510214924812
Epoch 19 Loss 10.00015114247799


In [37]:

def recommend_for_user(user_id, top_k=5):
    u = torch.tensor([user_enc.transform([user_id])[0]])
    res = []
    for idx, gid in enumerate(game_enc.classes_):
        if gid in df_ratings[df_ratings.user_id == user_id].game_id.values:
            continue
        g = torch.tensor([idx])
        f = game_features_tensor[g]
        score = model(u, g, f).item()
        name = df_games[df_games.game_id == gid].name.values[0]
        res.append((name, score))

    res.sort(key=lambda x: x[1], reverse=True)
    return res[:top_k]

print(recommend_for_user(1))


[('Wrecking Crew', 10.070301055908203), ('Eonwar', 9.572051048278809), ('Electro Havoc', 9.301798820495605), ('Retsnom', 8.807497024536133), ('Ben 10: 048 - Projectile Mod: Static Boom', 8.599088668823242)]
